In [ ]:
import zarr
import numpy as np

from numcodecs import Blosc, Zstd


def random_signal_matrix(self, buffer_size):
    return np.random.uniform(-1, 1, (buffer_size,)) + 1.0j * np.random.uniform(
        -1, 1, (buffer_size,)
    )


total_samples = 2**24
buffer_size = 2**18
assert total_samples % buffer_size == 0
timesteps = total_samples // buffer_size
chunk_size = 4096
data_points = 1024
filename = "testzarr"


f64_keys = [
    "system_timestamps",
    "gps_timestamps",
    "lat",
    "long",
    "heading",
    "avg_phase_diff",
    "rssi",
    "gain",
]


def v4rx_new_dataset(filename, n_receivers, compressor=None):
    z = zarr.open(
        filename,
        mode="w",
    )
    if compressor is None:
        compressor = Blosc(
            cname="zstd",
            clevel=1,
            shuffle=Blosc.BITSHUFFLE,
        )
    z.create_group("receivers")
    for receiver_idx in range(n_receivers):
        receiver_z = z["receivers"].create_group(f"r{receiver_idx}")
        receiver_z.create_dataset(
            "signal_matrix",
            shape=(timesteps, 2, buffer_size),
            chunks=(1, 1, 1024 * chunk_size),
            dtype="complex128",
            compressor=compressor,
        )
        for key in f64_keys:
            receiver_z.create_dataset(
                key,
                shape=(timesteps,),
                chunks=(1024 * chunk_size),
                dtype="float64",
                compressor=compressor,
            )
    return z


z = v4rx_new_dataset("testdata", 2)

In [ ]:
z.tree()

In [ ]:
z.receivers.r0

In [ ]:
z.create_group("test")
z.tree()

In [ ]:
!pip install ipytree
!pip install lmdb

In [ ]:
root = zarr.group(store=store, overwrite=True)

In [ ]:
import zarr

from numcodecs import Blosc, Zstd

total_samples = 2**24
buffer_size = 2**18
assert total_samples % buffer_size == 0
timesteps = total_samples // buffer_size
chunk_size = 4096
data_points = 1024
filename = "testzarr"


f64_keys = [
    "system_timestamps",
    "gps_timestamps",
    "lat",
    "long",
    "heading",
    "avg_phase_diff",
    "rssi",
    "gain",
]

store = zarr.LMDBStore("data/array.mdb")
z = zarr.open(
    store=store,
    mode="w",
)
compressor = Blosc(
    cname="zstd",
    clevel=1,
    shuffle=Blosc.BITSHUFFLE,
)
z.create_group("receivers")
for receiver_idx in range(n_receivers):
    receiver_z = z["receivers"].create_group(f"r{receiver_idx}")
    receiver_z.create_dataset(
        "signal_matrix",
        shape=(timesteps, 2, buffer_size),
        chunks=(1, 1, 1024 * chunk_size),
        dtype="complex128",
        compressor=compressor,
    )
    for key in f64_keys:
        receiver_z.create_dataset(
            key,
            shape=(timesteps,),
            chunks=(1024 * chunk_size),
            dtype="float64",
            compressor=compressor,
        )

In [ ]:
z.tree()

In [ ]:
repo_root = "/Users/miskodzamba/Dropbox/research/gits/spf/"
import sys

sys.path.append(repo_root)  # go to parent dir
from spf.dataset.spf_dataset import SessionsDatasetReal
from spf.dataset.v5_data import v5rx_new_dataset
from spf.utils import random_signal_matrix
from spf.dataset.v5_data import v5rx_2xf64_keys, v5rx_f64_keys, v5rx_new_dataset

import numpy as np

timesteps = 110
buffer_size = 2**13
fn = "./testdata5"
z = v5rx_new_dataset(
    fn,
    timesteps=timesteps,
    buffer_size=buffer_size,
    n_receivers=2,
)
for time_idx in range(timesteps):
    for receiver_idx in range(2):
        z.receivers[f"r{receiver_idx}"].signal_matrix[time_idx, :] = (
            random_signal_matrix(2 * buffer_size).reshape(2, buffer_size)
        )
        for k in v5rx_f64_keys:
            z.receivers[f"r{receiver_idx}"][k][time_idx] = np.random.rand()
        for k in v5rx_2xf64_keys:
            z.receivers[f"r{receiver_idx}"][k][time_idx, :] = np.random.rand()

In [ ]:
z0 = z["receivers/r1"].signal_matrix[:]

In [ ]:
import os

z.store.db.info(), os.path.getsize(fn)

In [ ]:
z.store.close()
z = None

In [ ]:
from spf.utils import zarr_shrink


zarr_shrink(fn)

In [ ]:
from spf.utils import zarr_open_from_lmdb_store


z = zarr_open_from_lmdb_store(fn)

In [ ]:
z = zarr_open_from_lmdb_store(fn)
z.store.db.info(), os.path.getsize(fn)

In [ ]:
z.store.close()
z = None

In [ ]:
z.store.close()

In [ ]:
import zarr

store = zarr.LMDBStore(fn, map_size=2**38, writemap=True, map_async=True)
z = zarr.open(store=store, mode="r")

In [ ]:
store.db.set_mapsize

In [ ]:
import zarr

store = zarr.LMDBStore(fn, map_size=2**38, writemap=True, map_async=True)
z = zarr.open(store=store, mode="r")
z.store.db.set_mapsize(1)

In [ ]:
z.store.db.info(), z.store.db.info(), os.path.getsize(fn)

In [ ]:
z.store.set_mapsize(1)
z.store.db.info(), z.store.db.info(), os.path.getsize(fn)